<a href="https://colab.research.google.com/github/Triwka/ml_homework/blob/main/kaggle_comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install geopy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import numpy as np
import pandas as pd

from geopy.distance import geodesic
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, average_precision_score
import joblib


#импортируем библиотеки matplotlib и seaborn для построения графиков
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [2]:
train = pd.read_csv('https://drive.google.com/uc?id=1pVqiPMX7L7bqviBCE-13_uA6ZfBCzOgw', sep=';')
test = pd.read_csv('https://drive.google.com/uc?id=1YtjNEL0Yye6tkW26vpDbfQQv3dZvpSYr', sep=';')
submission = pd.read_csv('https://drive.google.com/uc?id=1MTZnySU5mGdppCp9zq8mg-Of1I_rSai8', sep=';')

In [6]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level
0,1.0,0,cb0ecd5356125b546833239efdb154c2,2016-06-02 04:21:46,AMAZING STUDIO!! CONVENIENT TO ALL - G...,Gold Street,"['Cats Allowed', 'Dogs Allowed']",40.6969,7097709,-73.9830,79d1809d224bc7f26c719f247d6c8dc6,['https://photos.renthop.com/2/7097709_8f74305...,2575,257 Gold Street,low
1,1.0,2,4df46d9768a98df24200a2e7693370bd,2016-05-26 05:44:12,"Near NYU, Washington SQ Park, centrally locate...",Thompson Street,[],40.7264,7071860,-74.0013,e60b02e0cfd2eea7d777bb283d6f8d4d,['https://photos.renthop.com/2/7071860_32b196a...,3250,128 Thompson Street,high
2,1.0,0,0,2016-06-17 01:22:32,RENT STABILIZED! FULL TIME DOORMAN.LEXINGTON A...,East 35th Street,"['Doorman', 'Dogs Allowed', 'Cats Allowed']",40.7466,7174524,-73.9779,91a276fdb4c7ae5ff56fd0884b5121db,['https://photos.renthop.com/2/7174524_bde8df9...,2448,165 East 35th Street,low
3,1.0,1,78e50c94b33bb1ceeb5f4c8ff7455703,2016-06-03 05:26:40,WHY TO LIVE IN THIS LUXURIOUS BUILDING? WEEKDA...,West 42nd Street,"['Swimming Pool', 'Roof Deck', 'Dining Room', ...",40.7610,7104515,-73.9992,cbcd513212d592e6c717d27be14f46ac,['https://photos.renthop.com/2/7104515_bab0753...,3300,610 West 42nd Street,low
4,2.0,0,e87f1236b102798c1fa104a372800444,2016-04-12 04:10:28,Listed on the National Register of His...,Greenwich Street,"['Doorman', 'Fitness Center', 'No Fee']",40.7324,6859942,-74.0081,b209e2c4384a64cc307c26759ee0c651,['https://photos.renthop.com/2/6859942_16f88f7...,5450,666 Greenwich Street,low


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39481 entries, 0 to 39480
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   bathrooms        39481 non-null  float64
 1   bedrooms         39481 non-null  int64  
 2   building_id      39481 non-null  object 
 3   created          39481 non-null  object 
 4   description      38337 non-null  object 
 5   display_address  39376 non-null  object 
 6   features         39481 non-null  object 
 7   latitude         39481 non-null  float64
 8   listing_id       39481 non-null  int64  
 9   longitude        39481 non-null  float64
 10  manager_id       39481 non-null  object 
 11  photos           39481 non-null  object 
 12  price            39481 non-null  int64  
 13  street_address   39472 non-null  object 
 14  interest_level   39481 non-null  object 
dtypes: float64(3), int64(3), object(9)
memory usage: 4.5+ MB


In [10]:
train.duplicated().sum()

0

In [11]:
train.isna().sum()

bathrooms             0
bedrooms              0
building_id           0
created               0
description        1144
display_address     105
features              0
latitude              0
listing_id            0
longitude             0
manager_id            0
photos                0
price                 0
street_address        9
interest_level        0
dtype: int64

In [12]:
train.description = train.description.fillna('empty')
train = train.dropna()

## 1.Добавление новых фичей


1)Добавление фичей "количество атрибутов квартиры" и "количество фотографий"

In [23]:
train['num_features']=train['features'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
train['num_photos']=train['photos'].str.replace('[\[\]\']', '').str.split(', ').apply(len)

<ipython-input-23-6a11e9b84264>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train['num_features']=train['features'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
<ipython-input-23-6a11e9b84264>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['num_features']=train['features'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
<ipython-input-23-6a11e9b84264>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['num_photos']=train['photos'].str.replace('[\[\]\']', '').str.split(', ').apply(len)
<ipython-input-23-6a11e9b84264>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

2)ГИПОТЕЗА:популярость квартиры зависит от её удалённости до какого-либо объекта - добавление фичей "удалённость до какого-либо популярного объекта"

In [90]:
central_park = tuple([40.7736,-73.9727]) #Координаты центрального парка
time_square = tuple([40.7577,-73.9857])  #Координаты тайм-сквера
wall_street = tuple([40.7072,-74.0116])   #Координаты уолл-стрит

In [86]:
train['distance_central_park'] = train.latitude
train['distance_time_square'] = train.latitude
train['distance_wall_street'] = train.latitude

<ipython-input-86-6c4e55cd8bc2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['distance_central_park'] = train.latitude
<ipython-input-86-6c4e55cd8bc2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['distance_time_square'] = train.latitude
<ipython-input-86-6c4e55cd8bc2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [87]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,num_features,num_photos,distance_central_park,distance_time_square,distance_wall_street
0,1.0,0,cb0ecd5356125b546833239efdb154c2,2016-06-02 04:21:46,AMAZING STUDIO!! CONVENIENT TO ALL - G...,Gold Street,"['Cats Allowed', 'Dogs Allowed']",40.6969,7097709,-73.9830,79d1809d224bc7f26c719f247d6c8dc6,['https://photos.renthop.com/2/7097709_8f74305...,2575,257 Gold Street,low,2,10,40.6969,40.6969,40.6969
1,1.0,2,4df46d9768a98df24200a2e7693370bd,2016-05-26 05:44:12,"Near NYU, Washington SQ Park, centrally locate...",Thompson Street,[],40.7264,7071860,-74.0013,e60b02e0cfd2eea7d777bb283d6f8d4d,['https://photos.renthop.com/2/7071860_32b196a...,3250,128 Thompson Street,high,1,5,40.7264,40.7264,40.7264
2,1.0,0,0,2016-06-17 01:22:32,RENT STABILIZED! FULL TIME DOORMAN.LEXINGTON A...,East 35th Street,"['Doorman', 'Dogs Allowed', 'Cats Allowed']",40.7466,7174524,-73.9779,91a276fdb4c7ae5ff56fd0884b5121db,['https://photos.renthop.com/2/7174524_bde8df9...,2448,165 East 35th Street,low,3,1,40.7466,40.7466,40.7466
3,1.0,1,78e50c94b33bb1ceeb5f4c8ff7455703,2016-06-03 05:26:40,WHY TO LIVE IN THIS LUXURIOUS BUILDING? WEEKDA...,West 42nd Street,"['Swimming Pool', 'Roof Deck', 'Dining Room', ...",40.7610,7104515,-73.9992,cbcd513212d592e6c717d27be14f46ac,['https://photos.renthop.com/2/7104515_bab0753...,3300,610 West 42nd Street,low,16,6,40.7610,40.7610,40.7610
4,2.0,0,e87f1236b102798c1fa104a372800444,2016-04-12 04:10:28,Listed on the National Register of His...,Greenwich Street,"['Doorman', 'Fitness Center', 'No Fee']",40.7324,6859942,-74.0081,b209e2c4384a64cc307c26759ee0c651,['https://photos.renthop.com/2/6859942_16f88f7...,5450,666 Greenwich Street,low,3,8,40.7324,40.7324,40.7324


In [91]:
for i in range(train.shape[0]):
  train.distance_central_park.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),central_park).miles

for i in range(train.shape[0]):
  train.distance_time_square.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),time_square).miles

for i in range(train.shape[0]):
  train.distance_wall_street.iloc[i] = geodesic((train.latitude.iloc[i],train.longitude.iloc[i]),wall_street).miles

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [92]:
train.head()

,bathrooms,bedrooms,building_id,created,description,display_address,features,latitude,listing_id,longitude,manager_id,photos,price,street_address,interest_level,num_features,num_photos,distance_central_park,distance_time_square,distance_wall_street
0,1.0,0,cb0ecd5356125b546833239efdb154c2,2016-06-02 04:21:46,AMAZING STUDIO!! CONVENIENT TO ALL - G...,Gold Street,"['Cats Allowed', 'Dogs Allowed']",40.6969,7097709,-73.9830,79d1809d224bc7f26c719f247d6c8dc6,['https://photos.renthop.com/2/7097709_8f74305...,2575,257 Gold Street,low,2,10,5.320034,4.197741,1.661572
1,1.0,2,4df46d9768a98df24200a2e7693370bd,2016-05-26 05:44:12,"Near NYU, Washington SQ Park, centrally locate...",Thompson Street,[],40.7264,7071860,-74.0013,e60b02e0cfd2eea7d777bb283d6f8d4d,['https://photos.renthop.com/2/7071860_32b196a...,3250,128 Thompson Street,high,1,5,3.586089,2.309755,1.430961
2,1.0,0,0,2016-06-17 01:22:32,RENT STABILIZED! FULL TIME DOORMAN.LEXINGTON A...,East 35th Street,"['Doorman', 'Dogs Allowed', 'Cats Allowed']",40.7466,7174524,-73.9779,91a276fdb4c7ae5ff56fd0884b5121db,['https://photos.renthop.com/2/7174524_bde8df9...,2448,165 East 35th Street,low,3,1,1.882948,0.868433,3.243590
3,1.0,1,78e50c94b33bb1ceeb5f4c8ff7455703,2016-06-03 05:26:40,WHY TO LIVE IN THIS LUXURIOUS BUILDING? WEEKDA...,West 42nd Street,"['Swimming Pool', 'Roof Deck', 'Dining Room', ...",40.7610,7104515,-73.9992,cbcd513212d592e6c717d27be14f46ac,['https://photos.renthop.com/2/7104515_bab0753...,3300,610 West 42nd Street,low,16,6,1.639738,0.744031,3.768959
4,2.0,0,e87f1236b102798c1fa104a372800444,2016-04-12 04:10:28,Listed on the National Register of His...,Greenwich Street,"['Doorman', 'Fitness Center', 'No Fee']",40.7324,6859942,-74.0081,b209e2c4384a64cc307c26759ee0c651,['https://photos.renthop.com/2/6859942_16f88f7...,5450,666 Greenwich Street,low,3,8,3.395988,2.104669,1.748541


In [64]:
df.coor_tuple

0         40.6969 -73.983
1        40.7264 -74.0013
2        40.7466 -73.9779
3         40.761 -73.9992
4        40.7324 -74.0081
               ...       
39476     40.7446 -73.974
39477    40.7085 -74.0165
39478    40.5956 -73.7555
39479    40.7671 -73.9529
39480    40.7448 -73.9919
Name: coor_tuple, Length: 39375, dtype: object